# **Recurrent Neural Network - Analyse de sentiments**

Le but de cet exercice est la classification binaire (positive ou négative) de reviews en utilisant un RNN sur un dataset de commentaires IMDB que vous trouverez dans le dossier data.

In [31]:
# import des librairies
import string
from collections import Counter

## **1. Import des données et preprocessing**

1. Importer les 2 fichiers ".txt" de data/imdb
2. Vérifier ce qu'il y a dedans si c'est pas déjà fait...
3. Un peu de preprocessing de texte, allez on va chercher dans sa petite mémoire et sinon dans son gros ordinateur :
>- convertir en minuscules
>- retirer la ponctuation
>- créer une liste des reviews et déterminer combien il y en a. Pareil pour les labels. Inch'Allah y en aura autant.

In [1]:
# import des txt
with open('data/imdb/reviews.txt', 'r') as f:
    reviews_str = f.read()
with open('data/imdb/labels.txt', 'r') as f:
    labels_str = f.read()

In [6]:
# reviews
type(reviews_str), len(reviews_str), reviews_str[:150]

(str,
 33678267,
 'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching ')

In [7]:
# labels
type(labels_str), len(labels_str), labels_str[:150]

(str,
 225000,
 'positive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositi')

In [13]:
# netoyage des string
reviews_str = reviews_str.lower()
reviews_str = reviews_str.translate(str.maketrans('', '', string.punctuation))

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers   the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students  when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled          at           high  a classic line inspector i  m here to sack one of your teachers  student welcome to bromwell high  i expect that many adults of my age think that bromwell high is far fetched  what a pity that it isn  t   \nstory of a man who has unnatural feelings for a pig  starts out with a opening scene that is a terrific example of absurd comedy  a formal orchestra audience is turned into an insane  violent 

In [25]:
# split et nb de lables
rvws = reviews_str.split('\n')
labs = labels_str.split('\n')
len(rvws), len(labs)

(25001, 25001)

## **2. Tokenisation ou encodage**

Le but est de remplacer les mots des reviews par des entiers. Si vous voulez faire à votre sauce, vous y êtes encouragés !

Sinon, les quelques étapes décrites ci-dessous vous permettront de le faire :
>- compter l'ensemble des mots (vous pouvez utiliser `Counter` de la librairie `collections` qui est une des plus rapide dans ce domaine)
>- les trier par ordre décroissant d'occurrences
>- créer un dictionnaire `word_mapping` où les clés sont les mots et les valeurs l'entier associé (votre dico doit être `{'the': 1, 'and': 2, 'a': 3, 'of': 4,..., 'muppified': 74070, 'whelk': 74071, 'hued': 74072}}`). C'est volontaire que ça commence à 1 car on utilisera le "0" comme caractère spécial (il servira pour "remplir" les reviews les plus courtes afin qu'elles aient toutes la même taille...)
>- encoder les mots (c'est-à-dire les remplacer par l'entier qui les représente)
>- encoder les labels (ça c'est fastoche, on se débrouille)

In [38]:
# comptage des mots et tri par occurrence
words = reviews_str.split()
countwords = Counter(words)
sorted_words = sorted(countwords, key=countwords.get, reverse=True)

In [40]:
# dictionnaire de correspondance mot=entier
word_mapping = {w:i+1 for i,w in enumerate(sorted_words)}

In [47]:
# encodage des mots dans les reviews
rvws_num = [list(map(lambda w : word_mapping[w], rvw.split())) for rvw in rvws]
print(rvws_num[:2])

[[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23], [63, 4, 3, 125, 36, 47, 7472, 1395, 16, 3, 4181, 505, 45, 17, 3, 622, 134, 12, 6, 3, 1279, 457, 4, 1721, 207, 3, 10624, 7373, 300, 6, 667, 83, 35, 2116, 1086, 2989, 34, 1, 898, 46417, 4, 8, 13, 5096, 464, 8, 2656, 1721, 1, 221, 57, 17, 58, 794, 1297, 832, 228, 8, 43, 98, 123, 1469, 59, 147, 38, 1, 963, 142, 29, 667, 123, 1, 13584, 410, 61, 94, 177

In [49]:
# encodage des labels
labs_num = [1*(lab == 'positive') for lab in labs]
labs_num[:2]

[1, 0]

## **3. Longueur des séquences**

Il s'agit ici d'analyser la longueur des reviews pour déterminer éventuellement des outliers et choisir ce qu'on en fait. On va aussi "uniformiser" la longueur des séquence.

1. Avec la méthode de votre choix (graphique, stats desc...), étudier la longueur des reviews
2. Déterminer s'il y a des outliers et ce que vous souhaitez en faire (si vous les décidez de les supprimer, attention de bien supprimer aussi les labels correspondants...)
3. Ça a été évoqué un peu plus haut, on veut que nos reviews aient toutes la même taille pour faciliter l'entraînement du réseau. Par conséquent on va ajouter des 0 aux reviews les plus courtes et tronquer les reviews les plus longues (*padding/truncating*, si vous vous souvenez bien on a vu il y a peu le *zero-padding* dans un certain cas...bon ben c'est pareil).  
>- définir une fonction `trunc_pad(review_list, length)` :
>>- qui prend en paramètres la liste des reviews (chaque review étant encodée en une liste d'entiers) et une longueur donnée
>>- et qui retourne un array 2D avec (en ligne) les reviews trop longues tronquées et des 0 à gauche pour les reviews trop courtes. Avant de vous lancer et pour vous assurer d'avoir compris, quelles dimensions doit avoir votre array en sortie ?
>>- tester votre fonction avec une longueur fixée à 250 et afficher les 5 premières valeurs des 5 premières reviews. Vous devez obtenir ça:  
\[[    0     0     0     0     0]  
[    0     0     0     0     0]  
[22382    42 46418    15   706]  
[ 4505   505    15     3  3342]  
[    0     0     0     0     0]\]
>- maintenant, que c'est fait, je peux vous le dire, il y a une fonction dans `keras.preprocessing` qui permet de le faire. La trouver et comparer les temps d'éxecution des 2 fonctions.

## **4. Échantillons d'entraînement, de validation et de test**

Découper les données en train, validation et test sets de la manière qui vous plaira. Tant que c'est juste et cohérent bien sûr.  
Il faut qu'il y ait 20000 observations dans le train, 2500 dans le validation et 2500 dans le test. 

## **5. Petite paranthèse sur le _word embedding_**

Le [*word embedding*](https://fr.wikipedia.org/wiki/Word_embedding) consiste à transformer des mots sous forme de vecteurs de nombres. Bon ça on peut le faire relativement facilement, vous venez d'ailleurs de le faire avec la représentation du lexique des reviews en nombres entiers. Pour passer à un vecteur il suffirait juste de faire un one-hot-encoding.

Il y a plusieurs problèmes à cette solution (même si on l'utilise parfois dans du NLP simple) :
- la dimension de l'espace engendré
- l'absence totale de notion de similarité (avec la représentation one-hot, 2 mots qui n'ont rien à voir sont aussi différents que 2 mots tout à fait synonyme)
- le fait que les vecteurs contiennent quasiment que des 0 (sparse matrix)

Donc le *word embedding* s'attaque à ce problème avec pour objectifs de :
1. représenter les mots sous forme de vecteurs de nombres réels (et non entiers, on passe donc à un espace continu et plus discret)
2. conserver la notion de similarité c'est-à-dire que 2 vecteurs qui sont proches doivent représenter des mots "sémantiquement proches"
3. de les représenter dans un espace de plus petite dimension (en fonction de votre vocabulaire, soit le nombre de mots dans votre problème, ça peut aller très vite, généralement plusieurs milliers ou 10aines de milliers)

Le principe est de s'intéresser au contexte des mots, c'est-à-dire, quels mots sont associés ensemble en s'appuyant sur la co-occurrence.
Différents modèles de word embedding existent : [word2vec](https://fr.wikipedia.org/wiki/Word2vec), 

En pratique avec `keras`, que se passe-t-il lorsqu'on ajout une couche [*embedding*](https://keras.io/api/layers/core_layers/embedding/) ? Un exemple juste en dessous.

Un peu de visionnage pour y voir plus clair si ça vous intéresse :
- https://www.youtube.com/watch?v=Eku_pbZ3-Mw
- https://www.youtube.com/watch?v=oUpuABKoElw
- https://www.youtube.com/watch?v=5PL0TmQhItY

Et un peu de lecture :
- https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/
- https://towardsdatascience.com/why-do-we-use-embeddings-in-nlp-2f20e1b632d2
- https://towardsdatascience.com/word-embeddings-for-nlp-5b72991e01d4
- https://proceedings.neurips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf
- https://medium.com/rasa-blog/supervised-word-vectors-from-scratch-in-rasa-nlu-6daf794efcd8
- https://web.stanford.edu/~jurafsky/slp3/6.pdf

In [24]:
# on prend quelques phrases d'exemples pour illustrer le word embedding
t1 = "i hope to see you again"
t2 = "you wish to see me soon"
t3 = "i wish we will meet again"

## **6. Création, entraînement et évaluation du modèle**

Créer un premier modèle, que vous serez tout à fait libre et même cordialement conviés à améliorer par la suite, avec :
- une couche Embedding
- une couche LSTM